# **Mejor arquitecura con el canal verde**

En este notebook está la arquitectura que obtiene los mejores resultados en la búsqueda secuencial de hiperparámetros, y usando sólo el canal verde

In [1]:
import tensorflow as tf
from keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from tools import load_images_with_labels, evaluate_model

## Cargar datos

In [2]:
width = 540
height = 960
ruta_carpeta = '../data/burn_images/'

In [3]:
X, y = load_images_with_labels(path=ruta_carpeta, channels='green')

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

print('Shape of X_train:', X_train.shape)
print('Shape of X_val:', X_val.shape)
print('Shape of y_train:', y_train.shape)
print('Shape of y_val:', y_val.shape)

Shape of X_train: (180, 960, 540, 1)
Shape of X_val: (45, 960, 540, 1)
Shape of y_train: (180,)
Shape of y_val: (45,)


## Definir la arquitectura

In [5]:
cnn = tf.keras.Sequential([
    layers.Conv2D(32, kernel_size=(3,3), strides=(2,2), input_shape=(height, width, 1)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.Conv2D(32, kernel_size=(3,3), strides=(2,2)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.Conv2D(64, kernel_size=(3,3), strides=(2,2)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.GlobalAveragePooling2D(),

    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.4),
    
    layers.Dense(1),
    layers.Activation('sigmoid')
])

c:\Users\yamil\miniconda3\envs\ml\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 479, 269, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 479, 269, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 479, 269, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 239, 134, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 119, 66, 32)    │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 119, 66, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 119, 66, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 59, 33, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 29, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 29, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 29, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 8, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,801 (128.13 KB)

 Trainable params: 32,545 (127.13 KB)

 Non-trainable params: 256 (1.00 KB)

## Compilar y entrenar el modelo

In [ ]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Detiene el entrenamiento cuando el modelo deja de mejorar
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10, #Espera 10 épocas sin mejora antes de detener
    restore_best_weights=True #Restaura los pesos de la época con mejor val_loss
)

#Reduce el learning rate cuando el modelo se estanca
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, #Reduce el learning rate a la mitad cuando no hay mejora
    patience=5, #Espera 5 épocas sin mejora antes de reducir
    min_lr=1e-7 #Learning rate mínimo permitido
)

history = cnn.fit(X_train, y_train,
                  epochs=100, batch_size=32,
                  validation_data=(X_val, y_val),
                  callbacks=[early_stopping, reduce_lr])

## Evaluar el modelo

In [ ]:
evaluate_model(cnn, X_train, y_train, dataset='Train')

In [ ]:
evaluate_model(cnn, X_val, y_val, dataset='Validation')

## Guardar el modelo y el history de entrenamiento

In [ ]:
from keras import models
models.save_model(cnn, '../models/cnn_green.keras')

import pickle
with open('../models/history_cnn_green.pkl', 'wb') as f:
    pickle.dump(history.history, f)